In [1]:
import warnings

import numpy as np
import tensorflow as tf

import keras_spiking

warnings.simplefilter("ignore")
tf.get_logger().addFilter(lambda rec: "Tracing is expensive" not in rec.msg)

2024-02-23 22:06:25.135239: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 22:06:25.908102: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 22:06:25.912416: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-23 22:06:28.530867: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
inp = x = tf.keras.Input((None, 28, 28, 1))
x = tf.keras.layers.TimeDistributed(
    tf.keras.layers.Conv2D(filters=2, kernel_size=(7, 7))
)(x)
x = keras_spiking.SpikingActivation("relu")(x)
x = tf.keras.layers.TimeDistributed(
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3))
)(x)
x = keras_spiking.SpikingActivation("relu")(x)
x = tf.keras.layers.TimeDistributed(
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3))
)(x)
x = keras_spiking.SpikingActivation("relu")(x)
x = tf.keras.layers.TimeDistributed(
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3))
)(x)
x = keras_spiking.SpikingActivation("relu")(x)

x = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(x)
# x = tf.keras.layers.Dense(units=128)(x)
# x = keras_spiking.SpikingActivation("relu")(x)
x = tf.keras.layers.Dense(units=256)(x)
x = keras_spiking.SpikingActivation("relu")(x)
x = tf.keras.layers.Dense(units=10)(x)

model = tf.keras.Model(inp, x)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, 28, 28, 1)  0         
                             ]                                   
                                                                 
 time_distributed_7 (TimeDis  (None, None, 22, 22, 2)  100       
 tributed)                                                       
                                                                 
 spiking_activation_7 (Spiki  (None, None, 22, 22, 2)  0         
 ngActivation)                                                   
                                                                 
 time_distributed_8 (TimeDis  (None, None, 20, 20, 64)  1216     
 tributed)                                                       
                                                                 
 spiking_activation_8 (Spiki  (None, None, 20, 20, 64)  0  

In [7]:
energy = keras_spiking.ModelEnergy(model, example_data=np.ones((32, 32, 28, 28, 1)))
energy.summary(
    columns=(
        "name",
        "rate",
        "energy cpu",
        "energy arm",
        "energy gpu",
        "energy loihi",
        # "energy spinnaker",
        # "energy spinnaker2",
    ),
    timesteps_per_inference=100,
    dt=0.001,
    # print_warnings=False,
)

1/1 [==============================] - 3s 3s/step
Layer (type)                          |Rate [Hz]|J/inf (cpu)|J/inf (arm)|J/inf (gpu)|J/inf (loihi)
--------------------------------------|---------|-----------|-----------|-----------|-------------
input_3 (InputLayer)                  |        1|          0|          0|          0|            0
time_distributed_7 (TimeDistributed)  |        1|    0.00041|    4.3e-05|    1.4e-05|      1.3e-07
spiking_activation_7 (SpikingActivatio|     0.41|    8.3e-06|    8.7e-07|    2.9e-07|      7.8e-06
time_distributed_8 (TimeDistributed)  |     0.39|      0.004|    0.00041|    0.00014|      4.9e-07
spiking_activation_8 (SpikingActivatio|     0.37|    0.00022|    2.3e-05|    7.7e-06|      0.00021
time_distributed_9 (TimeDistributed)  |     0.18|       0.41|      0.043|      0.014|      2.3e-05
spiking_activation_9 (SpikingActivatio|     0.91|    0.00071|    7.5e-05|    2.5e-05|      0.00067
time_distributed_10 (TimeDistributed) |     0.46|        2.